In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta


%matplotlib inline  

In [2]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a'
api_key = 'c81352430e2fe3c941faf0814227562b'

data

In [3]:
''' Колонки '''

ticker = 'AI'

# tickers_list = ['AXP', 'INTC']
# tickers_list = ['ADBE', 'COP', 'AMAT']
# tickers_list = ['QTWO', 'VRSN', 'CACI', 'MDB', 'NOK', 'IDCC', 'TWOU', 'ESTC', 'SQ', 'OLED', 'AFRM']
# tickers_list = ['UAA', 'HBAN', 'HPE', 'IVZ', 'KEY', 'RF', 'BEN', 'WMB', 'DAL', 'GM', 'WRK', 'WFC', 'EBAY', 'VTR', 'AIG']
tickers_list =  ['MU', 'MET', 'NTAP', 'SYY', 'BXP', 'MNST', 'XOM', 'PYPL', 'DIS', 'TROW', 'HLT', 'QCOM', 'CVX', 'AAPL', 'HON', 'UPS', 'LOW', 'ACN', 'SNPS', 'LRCX', 'BKNG']

required_year = '2022'

''' время '''
stime='2015-01-01'
period = '1mo'  # '1d' '1wk'

In [4]:
def change_percent_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                if y != len(v):
                    if v[y] < 0 and v[y+1] > 0:
                        l.append('Minus')
                    elif v[y] > 0 and v[y+1] < 0:
                        l.append('Plus')
                    else:
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv) 
                else:
                    vv = 0
                    l.append(vv)
                    # print(vv)
            except:
                l.append(0)
        df[f'change % {x[i]}'] = l
    return df

In [5]:
def one_stock_analyse(ticker, api_key, required_year):
    try:
        ''' запрос к yahoo за ценами '''
        def get_data_from_ticker(tick, start_time='2017-01-01', interval='1mo'):
            ticker = yf.Ticker(tick)

            df = ticker.history(start=start_time, end=None, interval=interval)
            x = pd.DataFrame(df)
            x.rename(columns={"Close": tick}, inplace=True)
            z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
            return z

        df = get_data_from_ticker(ticker)

        ''' получаем цену на период отчета и дату отчета'''
        # income_statement = fa.income_statement(ticker, api_key)
        # accepted_date = income_statement.T['acceptedDate'][0][:10]
        # df_report = get_data_from_ticker(ticker, accepted_date)
        # report_price = df_report.iloc[0,0]

        ''' получаем актуальную цену акции '''
        today = datetime.date.today()
        sdate = today - relativedelta(months=3)
        sdate = str(sdate)
        df_current_price = get_data_from_ticker(ticker, sdate, '1d')
        current_price = df_current_price.iloc[-1,0]

        ''' меняем индекс для корреляции с df ключевых метрик '''
        def change_index(df):
            df_index = df.index.to_list()
            for i in range(len(df_index)):
                t = df_index[i].date()
                df_index[i] = t.strftime('%Y-%m-%d')
            df['Date_index'] = df_index
            df = df.set_index('Date_index')
            return df

        df = change_index(df)

        ''' получаем income statment для анализа '''
        def get_key_metricks(ticker, api_key, required_year):
            key_metrics = fa.income_statement(ticker, api_key, period='annual')
            df_key_metr = key_metrics.T
            df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
            return df_key_metr



        df_key_metr = get_key_metricks(ticker, api_key, required_year)


        if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
            last_year = True
        else:
            last_year = False



        def get_index(df):
            index_years = df.index.to_list()
            return index_years

        index_years = get_index(df_key_metr)



        def get_price_from_df(df, list_dates):
            new_list = []
            for i in range(len(list_dates)):
                x = int(list_dates[i])
                x += 1
                x = str(x)
                try:
                    r = df.loc[f'{x}-01-01']
                    r = float(r)
                    new_list.append(r)
                except:
                    new_list.append(0)

            return new_list

        stock_year_prices = get_price_from_df(df, index_years)


        df_key_metr[ticker] = list(stock_year_prices)
        df_key_metr.drop('period', axis=1, inplace=True)

        def change_percent_all_columns(df):
            x = df.columns.to_list()
            for i in range(len(x)):
                v = df[x[i]].to_list()
                l = []
                for y in range(len(v)):
                    try:
                        if y != len(v):
                            vv = (v[y] / (v[y+1] / 100)) - 100
                            l.append(vv)
                        else:
                            vv = 0
                            l.append(vv)
                    except:
                        l.append(0)
                df[f'change % {x[i]}'] = l
            return df

        df_with_changes = change_percent_all_columns(df_key_metr)

        # if df_with_changes[f'change % {ticker}'][0] > df_with_changes[f'change % {ticker}'][1] and df_with_changes['change % debtToAssets'][0] < df_with_changes['change % debtToAssets'][1]:
        #     better_less_last = "Short"
        if df_with_changes['change % costAndExpenses'][0] < 0 and df_with_changes['change % costAndExpenses'][1] < 0 and df_with_changes['change % grossProfit'][0] < 0 and df_with_changes['change % grossProfit'][0] != 'Plus' and df_with_changes['change % eps'][0] < 140 and df_with_changes['change % eps'][0] != 'Plus':
            better_less_last = "Long"
        else:
            better_less_last = False



        ''' получаем цену во время отчета'''
        df_report = get_data_from_ticker(ticker, df_with_changes['acceptedDate'][0][:10])
        report_price = df_report.iloc[0,0]

        ''' получаем изменение цены '''
        change_price = current_price / (report_price/100) - 100

        if last_year == True and better_less_last != False:
            # df_last = df_with_changes.reindex(index=df_with_changes.index[::-1])
            # r = [ticker, accepted_date, report_price, current_price, change_price, df_with_changes['change % debtToAssets'][0], better_less_last]
            r = [ticker, df_with_changes['acceptedDate'][0][:10], report_price, current_price, change_price, df_with_changes['change % debtToAssets'][0], better_less_last]
        else:
            r = None 

        print(r)
    except:
        r = None
    return r
    # return df_report
    # return accepted_date

results of stock's data

In [6]:
def get_results_da_price(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        r = one_stock_analyse(tickers_list[i], api_key, required_year)
        if r == None:
            pass
        else:
            l.append(r)
    return l
  
df = pd.DataFrame(get_results_da_price(tickers_list, api_key, required_year), columns = ['Ticker', 'Accepted_Date', 'Price', 'Current Price', 'Change % price', 'change % debtToAssets', 'Side'])
df

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Empty DataFrame
Columns: [Ticker, Accepted_Date, Price, Current Price, Change % price, change % debtToAssets, Side]
Index: []

results

In [7]:
l  = []
for i in range(len(df['Change % price'])):
    if df['Side'][i] == 'Long':
        r = df['Change % price'][i] - df['change % debtToAssets'][i]
        r = r *(-1) if r < 0 else r
        l.append(r)
    elif df['Side'][i] == 'Short':
        r = df['Change % price'][i] - df['change % debtToAssets'][i]
        r = r *(-1) if r > 0 else r

        l.append(r)
    else:
        l.append(0)

df['Differense'] = l

In [8]:
df

Empty DataFrame
Columns: [Ticker, Accepted_Date, Price, Current Price, Change % price, change % debtToAssets, Side, Differense]
Index: []

add_key_metrics

In [9]:

# def add_key_metrics(df):
#     v = 'revenuePerShare'
#     v1 = 'netIncomePerShare'
#     v2 = 'inventoryTurnover'
#     v3 = 'capexPerShare'
#     v4 = 'cashPerShare'
#     v5 = 'bookValuePerShare'
#     v6 = 'daysPayablesOutstanding'
#     v7 = 'receivablesTurnover'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#         else:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_key_metrics(df)

In [10]:
'''
revenuePerShare < -10 = повышает шансы к лонгу
netIncomePerShare == Minus = повышает шансы к Лонгу
netIncomePerShare == Plus = повышает шансы к Шорту
bookValuePerShare > 100 = повышает шансы к Лонгу
capexPerShare > 100 = повышает шансы к Лонгу
daysPayablesOutstanding > 50 = повышает шансы к Лонгу
'''

'\nrevenuePerShare < -10 = повышает шансы к лонгу\nnetIncomePerShare == Minus = повышает шансы к Лонгу\nnetIncomePerShare == Plus = повышает шансы к Шорту\nbookValuePerShare > 100 = повышает шансы к Лонгу\ncapexPerShare > 100 = повышает шансы к Лонгу\ndaysPayablesOutstanding > 50 = повышает шансы к Лонгу\n'

add_income_statement

In [11]:
# def add_income_statement(df):
#     v = 'revenue'
#     v1 = 'costOfRevenue'
#     v2 = 'grossProfit'
#     v3 = 'researchAndDevelopmentExpenses'
#     v4 = 'costAndExpenses'
#     v5 = 'ebitda'
#     v6 = 'operatingIncome'
#     v7 = 'eps'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#         else:
#             key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_income_statement(df)

In [12]:
# df_with_parameters